In [1]:
!unzip /content/trained-model.zip


Archive:  /content/trained-model.zip
  inflating: trained-model/adapter_config.json  
  inflating: trained-model/adapter_model.bin  
  inflating: trained-model/README.md  


In [2]:
!pip install datasets -qqq #Es para cargar el dataset desde hf y desde ahi aplicar las consultas de PYSQL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

# Carga el dataset desde Hugging Face
dataset = load_dataset('ManeAI31416/EjemploCalificaciones')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
!pip install -qqq -U accelerate==0.23.0 peft==0.5.0 bitsandbytes==0.41.1 transformers==4.34 trl==0.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [5]:
import torch
from transformers import (
    AutoModelForCausalLM,
    CodeLlamaTokenizer,
    default_data_collator,
    Trainer,
    TrainingArguments,
    TrainerCallback,
    BitsAndBytesConfig,
    AutoTokenizer,
)
from contextlib import nullcontext
from tqdm import tqdm
import json
import copy
import datasets
from peft import LoraConfig, PeftConfig
from transformers import default_data_collator, Trainer

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [6]:
model_name = "codellama/CodeLlama-7b-Instruct-hf"

# dataset_id = "HuggingFaceH4/CodeAlpaca_20K"

# compute_dtype = getattr(torch, "float16")

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtyp=torch.bfloat16
# )

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    load_in_8bit=True
#     quantization_config=bnb_config,
)

model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
### PRUEBAS ###

In [53]:
evalua=""
evalua =  "CREATE TABLE escuela (\n    CUATRIMESTRE INT,\n    TURNO VARCHAR(255),\n    IDCARRERAGRUPO VARCHAR(255),\n    CARRERA VARCHAR(255),\n    NO INT,\n    PROFESOR VARCHAR(255),\n    MATERIA VARCHAR(255),\n    escuela VARCHAR(255),\n    DIA VARCHAR(255),\n    CANTIDAD  INT,\n    GRUPO VARCHAR(255),\n    GRUPO_EN_SISTEMA VARCHAR(255),\n    AULA VARCHAR(255)\n)\n    \n-- Using valid SQLite, answer the following questions for the tables provided above.\n\n-- "
evalua

'CREATE TABLE escuela (\n    CUATRIMESTRE INT,\n    TURNO VARCHAR(255),\n    IDCARRERAGRUPO VARCHAR(255),\n    CARRERA VARCHAR(255),\n    NO INT,\n    PROFESOR VARCHAR(255),\n    MATERIA VARCHAR(255),\n    escuela VARCHAR(255),\n    DIA VARCHAR(255),\n    CANTIDAD  INT,\n    GRUPO VARCHAR(255),\n    GRUPO_EN_SISTEMA VARCHAR(255),\n    AULA VARCHAR(255)\n)\n    \n-- Using valid SQLite, answer the following questions for the tables provided above.\n\n-- '

In [8]:
!pip install ffmpeg-python -qqq
!pip install SpeechRecognition -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 43.1 MB/s eta 0:00:00


In [54]:
# Pregunta al usuario si quiere escribir el texto o usar la voz
opcion = input("¿Quieres escribir el texto (escribe '1') o usar la voz (escribe '2')? ")

if opcion == '1':
    # Si el usuario elige escribir el texto, usa la entrada estándar
    mi_prompt = input("Escribe aquí tu prompt personalizado.")
elif opcion == '2':
    from IPython.display import HTML, Audio
    from google.colab.output import eval_js
    from base64 import b64decode
    import numpy as np
    from scipy.io.wavfile import read as wav_read
    import io
    import ffmpeg

    AUDIO_HTML = """
    <script>
    var my_div = document.createElement("DIV");
    var my_p = document.createElement("P");
    var my_btn = document.createElement("BUTTON");
    var t = document.createTextNode("Press to start recording");

    my_btn.appendChild(t);
    //my_p.appendChild(my_btn);
    my_div.appendChild(my_btn);
    document.body.appendChild(my_div);

    var base64data = 0;
    var reader;
    var recorder, gumStream;
    var recordButton = my_btn;

    var handleSuccess = function(stream) {
      gumStream = stream;
      var options = {
        //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
        mimeType : 'audio/webm;codecs=opus'
        //mimeType : 'audio/webm;codecs=pcm'
      };
      //recorder = new MediaRecorder(stream, options);
      recorder = new MediaRecorder(stream);
      recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
          base64data = reader.result;
          //console.log("Inside FileReader:" + base64data);
        }
      };
      recorder.start();
      };

    recordButton.innerText = "Recording... press to stop";

    navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


    function toggleRecording() {
      if (recorder && recorder.state == "recording") {
          recorder.stop();
          gumStream.getAudioTracks()[0].stop();
          recordButton.innerText = "Saving the recording... pls wait!"
      }
    }

    // https://stackoverflow.com/a/951057
    function sleep(ms) {
      return new Promise(resolve => setTimeout(resolve, ms));
    }

    var data = new Promise(resolve=>{
    //recordButton.addEventListener("click", toggleRecording);
    recordButton.onclick = ()=>{
    toggleRecording()

    sleep(2000).then(() => {
      // wait 2000ms for the data to be available...
      // ideally this should use something like await...
      //console.log("Inside data:" + base64data)
      resolve(base64data.toString())

    });

    }
    });

    </script>
    """

    def get_audio():
      display(HTML(AUDIO_HTML))
      data = eval_js("data")
      binary = b64decode(data.split(',')[1])

      process = (ffmpeg
        .input('pipe:0')
        .output('pipe:1', format='wav')
        .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
      )
      output, err = process.communicate(input=binary)

      riff_chunk_size = len(output) - 8
      # Break up the chunk size into four bytes, held in b.
      q = riff_chunk_size
      b = []
      for i in range(4):
          q, r = divmod(q, 256)
          b.append(r)

      # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
      riff = output[:4] + bytes(b) + output[8:]

      sr, audio = wav_read(io.BytesIO(riff))

      return audio, sr

    # Si el usuario elige usar la voz, graba el audio
    audio, sr = get_audio()

    # Guarda el audio en formato wav
    from scipy.io.wavfile import write
    write('/content/input.wav', sr, audio)

    # Convierte el audio a texto
    import speech_recognition as sr
    r = sr.Recognizer()
    with sr.AudioFile("/content/input.wav") as source:
        audio_data = r.record(source)
        text = r.recognize_google(audio_data, language='es-ES')

    # Imprime el texto convertido
    print("Texto convertido del audio: ", text)

    # Usa el texto convertido como prompt
    mi_prompt = text



¿Quieres escribir el texto (escribe '1') o usar la voz (escribe '2')? 1
Escribe aquí tu prompt personalizado.Dime todos los profesores que imparten la materia de mercadotecnia


In [ ]:
#mi_prompt = input ("Escribe aquí tu prompt personalizado.")

In [55]:
extra='\n\nSELECT'

In [56]:
evalua=evalua+mi_prompt+extra
evalua

'CREATE TABLE escuela (\n    CUATRIMESTRE INT,\n    TURNO VARCHAR(255),\n    IDCARRERAGRUPO VARCHAR(255),\n    CARRERA VARCHAR(255),\n    NO INT,\n    PROFESOR VARCHAR(255),\n    MATERIA VARCHAR(255),\n    escuela VARCHAR(255),\n    DIA VARCHAR(255),\n    CANTIDAD  INT,\n    GRUPO VARCHAR(255),\n    GRUPO_EN_SISTEMA VARCHAR(255),\n    AULA VARCHAR(255)\n)\n    \n-- Using valid SQLite, answer the following questions for the tables provided above.\n\n-- Dime todos los profesores que imparten la materia de mercadotecnia\n\nSELECT'

In [57]:

import re

In [58]:
model.eval()
model_input = tokenizer(evalua, return_tensors="pt").to("cuda")

with torch.no_grad():
    output = tokenizer.decode(model.generate(**model_input, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True)

match = re.search(r'(SELECT.*?)--', output, re.DOTALL)
if match:
    text_after_select = match.group(1)
else:
    text_after_select = output

print(text_after_select)



SELECT PROFESOR FROM escuela WHERE MATERIA = 'MERCADOTECNIA';




In [14]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [15]:
!pip install gtts -qqq

In [16]:
import io
import sys
from gtts import gTTS
import IPython.display as ipd

In [17]:
# Genera el objeto de voz
tts = gTTS(text=text_after_select, lang='es')

# Guarda el objeto de voz en un archivo
tts.save('output.mp3')

# Reproduce el archivo de audio
ipd.Audio('output.mp3', autoplay=True)


In [59]:
import numpy as np
import pandas as pd

escuela = pd.read_csv('/content/EjemploCalificaciones2.csv')



import sqlalchemy
from sqlalchemy import create_engine
print("version",sqlalchemy.__version__)




!pip install pandasql -qqq
import pandasql as ps

version 2.0.29


In [60]:
# Tu consulta SQL
consulta_sql = text_after_select

# Usa pandasql para ejecutar la consulta SQL en tu DataFrame
resultado = ps.sqldf(consulta_sql, locals())

# Imprime el resultado
print(resultado)


                        PROFESOR
0          ARELLANO MOLINA EDITH
1   GONZALEZ MATA EMANUEL DAMIAN
2            CRUZ ESPARZA BERTHA


In [61]:
import io
import sys
from gtts import gTTS
import IPython.display as ipd

In [65]:
# Convierte el DataFrame a una cadena de texto
texto = resultado.to_string(index=False)

# Genera el objeto de voz
tts = gTTS(text=texto, lang='es')

# Guarda el objeto de voz en un archivo
tts.save('output.mp3')

# Reproduce el archivo de audio
ipd.Audio('output.mp3', autoplay=True)
